# Prepare the data

In [1]:
import sys,os
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [ ]:
# all the gene-pathway annotation
df = pd.read_csv('Sly_pathway_annotation_20180827_with_expression_5_members.txt',header=None,index_col=None,sep='\t')
# gene-pathway annotation for only non-overlapping genes in 85 pathways
df_uni = pd.read_csv('Sly_pathway_annotation_20190117_with_expression_5_members_nonoverlapping.txt',header=None,index_col=None,sep='\t')
df.columns = ['Pathway','Gene']
pathway = df_uni[0].unique()
# to get back genes with multiple pathway annotation in only those 85 pathways
df_85 = df[df['Pathway'].isin(pathway)]
gene = df_85['Gene'].unique()
df_85.index = df_85['Gene']
df_85 = df_85.drop(['Gene'],axis=1)
df_85.columns = ['Pathway']

### make the multiple labels, using OneHotEncoder

In [ ]:
cat_encoder = OneHotEncoder() # or cat_encoder = OneHotEncoder(sparse=False)
df_85_1hot = cat_encoder.fit_transform(df_85)
labels = pd.DataFrame(df_85_1hot.toarray()) # or labels = pd.DataFrame(df_85_1hot)
labels.columns = cat_encoder.categories_[0]
labels.index = df_85.index
# group rows with same index ID, add up the 1s
gp = labels.groupby(level=0).sum()

### parse the expression matrix, here FCs in all combination were used

In [ ]:
exp = pd.read_csv('Results_Fold_changes_all_combination_Sly_20180124.txt',header=0,index_col=None,sep='\t')
exp_85 = exp[exp['gene'].isin(gene)]
exp_85.index = exp_85['gene']
exp_85 = exp_85.drop('gene',axis=1)
res = pd.concat([gp,exp_85],axis=1)
res.to_csv('Multilabel_85_pathways_matrix_all_FC.txt',index=True, header=True,sep="\t")